# GANs Practical Assignment Solutions
This notebook contains solutions for all 5 tasks.

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import numpy as np

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

## Task 1: Basic GAN Modification
Modify GAN for 32x32 images.

In [ ]:
# Task 1: Basic GAN for 32x32 Images
latent_dim = 100
batch_size = 64
learning_rate = 0.0002

transform32 = transforms.Compose([
    transforms.Resize(32),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])
train_loader32 = DataLoader(datasets.MNIST(root="./data", train=True, transform=transform32, download=True), batch_size=batch_size, shuffle=True)

class Generator32(nn.Module):
    def __init__(self, latent_dim):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(latent_dim, 256), nn.ReLU(True),
            nn.Linear(256, 512), nn.ReLU(True),
            nn.Linear(512, 1024), nn.Tanh()
        )
    def forward(self, z): return self.net(z).view(z.size(0), 1, 32, 32)

class Discriminator32(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(1024, 512), nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, 256), nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(256, 1), nn.Sigmoid()
        )
    def forward(self, img): return self.net(img.view(img.size(0), -1))

G32 = Generator32(latent_dim).to(device)
D32 = Discriminator32().to(device)
print("Task 1 components initialized.")

## Task 2: DCGAN Filter Experiment
Experimenting with 32 and 64 filters in the Discriminator.

In [ ]:
# Task 2: DCGAN Filter Experiment (32 and 64 filters)
class DiscriminatorExp(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv2d(1, 32, 4, 2, 1), # Reduced from 64
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(32, 64, 4, 2, 1), # Reduced from 128
            nn.BatchNorm2d(64),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Flatten(),
            nn.Linear(64 * 7 * 7, 1),
            nn.Sigmoid()
        )
    def forward(self, img): return self.net(img)

D_exp = DiscriminatorExp().to(device)
print("Task 2: Modified DCGAN Discriminator initialized.")


## Task 3: Conditional Generation Test
Implementing a Conditional GAN for MNIST digits.

In [ ]:
# Task 3: Conditional Generation (MNIST)
class ConditionalGenerator(nn.Module):
    def __init__(self, latent_dim, num_classes):
        super().__init__()
        self.label_emb = nn.Embedding(num_classes, num_classes)
        self.project = nn.Sequential(nn.Linear(latent_dim + num_classes, 128 * 7 * 7), nn.ReLU(True))
        self.net = nn.Sequential(
            nn.Unflatten(1, (128, 7, 7)),
            nn.ConvTranspose2d(128, 64, 4, 2, 1), nn.BatchNorm2d(64), nn.ReLU(True),
            nn.ConvTranspose2d(64, 1, 4, 2, 1), nn.Tanh()
        )
    def forward(self, z, labels):
        label_vec = self.label_emb(labels)
        x = torch.cat((z, label_vec), dim=1)
        return self.net(self.project(x))

def show_mnist_conditional(G, device):
    G.eval()
    z = torch.randn(10, 100, device=device)
    labels = torch.arange(10, device=device)
    with torch.no_grad():
        imgs = G(z, labels).squeeze().cpu().numpy()
    fig, axes = plt.subplots(1, 10, figsize=(15, 2))
    for i in range(10):
        axes[i].imshow(imgs[i], cmap='gray')
        axes[i].set_title(str(i))
        axes[i].axis('off')
    plt.show()

G_cond = ConditionalGenerator(100, 10).to(device)
print("Task 3: Conditional Generator ready.")
# show_mnist_conditional(G_cond, device) # Call after training


## Task 4: Change Dataset
Adapting the pipeline for Fashion-MNIST.

In [ ]:
# Task 4: Change Dataset to Fashion-MNIST
f_transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
fashion_train = datasets.FashionMNIST(root="./data", train=True, transform=f_transform, download=True)
fashion_loader = DataLoader(fashion_train, batch_size=64, shuffle=True)
print(f"Task 4: Fashion-MNIST loaded ({len(fashion_train)} images).")


## Task 5: Conditional Fashion Generator
Generating specific Fashion-MNIST items.

In [ ]:
# Task 5: Conditional Fashion-MNIST Generator
# Can reuse ConditionalGenerator class from Task 3
G_fashion = ConditionalGenerator(100, 10).to(device)

def generate_fashion(label_idx, G, device):
    labels_map = {0: 'T-shirt', 1: 'Trouser', 2: 'Pullover', 3: 'Dress', 4: 'Coat', 
                  5: 'Sandal', 6: 'Shirt', 7: 'Sneaker', 8: 'Bag', 9: 'Ankle Boot'}
    G.eval()
    z = torch.randn(1, 100, device=device)
    label = torch.tensor([label_idx], device=device)
    with torch.no_grad():
        img = G(z, label).squeeze().cpu().numpy()
    plt.imshow(img, cmap='gray')
    plt.title(f"Generated: {labels_map[label_idx]}")
    plt.axis('off')
    plt.show()

print("Task 5: Fashion-MNIST components ready.")
# generate_fashion(0, G_fashion, device) # Call after training
